In [44]:
import pandas as pd
import Functions
%run Functions.py
cwd = '/data/co2flux/common/mdomhoef/Oslo/Oslo_analysis/'
StationDataPath = cwd + 'data/HH_Data_small/'
year = 2020
years = [2017, 2018, 2019, 2020]


# lambdaGPP 	radZero 	alpha 	beta 	Tmin 	Tmax 	Topt
parameters_wrf = {1:[275.4595, 0.22577703, 0.28773167, -1.09316696, 0., 40, 20.],
              2:[254.4188, 0.21489270, 0.18056630, 0.83641734, 0., 40, 20.,],
              3:[446.0888,0.16293380,0.24447911, -0.48669162, 0., 40, 20.],
              4:[70.3829,0.29311134,0.05464646, -0.12080592, 2., 40, 20.],
              5:[682.0, 0.1141, 0.0049, 0.0000, 2., 40, 20.],
              6:[1132.2, 0.08626603, 0.09231632,0.28788863, 5., 40, 22.],
              7:[527.9303, 0.11930965, 0.1245603, 0.01743361, 2., 40 ,18.],
              8:[0.00, 0.00, 0.00, 0, 0., 40, 0.]
             } 

### 1. Combine and compare parameters for same PFT 
#### 1.1 Mean over all stations (specific year) 

In [31]:
df_morris_0, df_gpp_0, df_reco_0 = load_params_as_df(year, 0)

In [32]:
mean_morris = df_morris_0.mean()
mean_gpp = df_gpp_0.mean()
mean_reco = df_reco_0.mean()

In [5]:
mean_gpp

morr_RMSE         1.948630
new_RMSE_gd       1.888137
new_RMSE_ev       1.881409
lambdaGPP_gd      0.317058
radZero_gd      229.999964
lambdaGPP_ev      0.332826
radZero_ev      209.034990
dtype: float64

In [37]:
# predict reco with all 4 versions 
#compare to 0riginal 
snames = df_morris_0.index
df_mean_reco = pd.DataFrame(index = snames, columns = ['morr_RMSE', 'gd_RMSE','ev_RMSE', 'wrf_RMSE'])
for sitename in snames: 
    df = get_station_observation_year(StationDataPath, sitename, year)
    reco_or = df['RECO_DT_VUT_REF']    
    t = df['TA_F']
    
    reco_morris = reco(mean_morris.alpha, mean_morris.beta, t)
    reco_gd = reco(mean_reco.alpha_gd, mean_reco.beta_gd, t) 
    reco_ev = reco(mean_reco.alpha_ev, mean_reco.beta_gd,t)
    reco_wrf = reco(parameters_wrf[1][2], parameters_wrf[1][3], t)
    
    df_mean_reco.loc[sitename]['morr_RMSE'] = m_rmse(reco_or, reco_morris)
    df_mean_reco.loc[sitename]['gd_RMSE'] = m_rmse(reco_or, reco_gd)
    df_mean_reco.loc[sitename]['ev_RMSE'] =m_rmse(reco_or, reco_ev)
    df_mean_reco.loc[sitename]['wrf_RMSE'] = m_rmse(reco_or, reco_wrf) 
    
    # print('Morris :', rmse_morris, 'GD: ', rmse_gd, 'EV:  ', rmse_ev, 'wrf: ', rmse_wrf)

In [38]:
#the RMSE of RECO of all stations when mean was taken of alpha beta (for each method) for all stations 
df_mean_reco.mean()

morr_RMSE    2.221799
gd_RMSE      1.835945
ev_RMSE      1.828289
wrf_RMSE     3.077414
dtype: float64

In [9]:
# predict GPP #compare to 0riginal 
df_mean_gpp = pd.DataFrame(index = snames, columns = ['morr_RMSE', 'gd_RMSE','ev_RMSE', 'wrf_RMSE'])
station_data = pd.read_csv('Oslo_analysis/Station_evi_lswi_temp_rad_'+str(year)+'.csv', header = 0, index_col=0)

for sitename in snames: 
    df = get_station_observation_year(StationDataPath, sitename, year)
    lambdaGPP, Tscale, Wscale, Pscale, EVI, Rad, radZero, GPP_or = load_data_gpp(df_morris_0, station_data, sitename, year, StationDataPath)
    
    gee_morris = GEE_calculate(mean_morris.lambdaGPP, Tscale, Wscale, Pscale, EVI, Rad, mean_morris.radZero)
    gee_gd = GEE_calculate(mean_gpp.lambdaGPP_gd, Tscale, Wscale, Pscale, EVI, Rad, mean_gpp.radZero_gd)
    gee_ev = GEE_calculate(mean_gpp.lambdaGPP_ev, Tscale, Wscale, Pscale, EVI, Rad, mean_gpp.radZero_ev)
    gee_wrf = GEE_calculate(parameters_wrf[1][0], Tscale, Wscale, Pscale, EVI, Rad, parameters_wrf[1][1])

    df_mean_gpp.loc[sitename]['morr_RMSE'] = m_rmse(GPP_or, gee_morris)
    df_mean_gpp.loc[sitename]['gd_RMSE'] = m_rmse(GPP_or, gee_gd)
    df_mean_gpp.loc[sitename]['ev_RMSE'] =m_rmse(GPP_or, gee_ev)
    df_mean_gpp.loc[sitename]['wrf_RMSE'] = m_rmse(GPP_or, gee_wrf) 

In [10]:
df_mean_gpp.mean()

morr_RMSE    2.044255
gd_RMSE      2.054533
ev_RMSE      2.046297
wrf_RMSE     5.727492
dtype: float64

In [16]:
for year in years:
    df_preds, df_mean_nee = predict_NEE(year, StationDataPath, parameters_wrf, 0)
    print(year)
    print(df_mean_nee.mean())
    print('min: ',df_mean_nee.mean().idxmin())
    print('max: ',df_mean_nee.mean().idxmax())
    

2017
morr_RMSE     3.380155
gd_RMSE       3.483282
ev_RMSE       3.410174
wrf_RMSE      6.009448
morr_RMSEc    2.976334
gd_RMSEc      3.113384
ev_RMSEc      2.996041
wrf_RMSEc     5.756492
dtype: float64
min:  morr_RMSEc
max:  wrf_RMSE
2018
morr_RMSE     2.662658
gd_RMSE       2.641553
ev_RMSE       2.593869
wrf_RMSE      6.445116
morr_RMSEc    2.298196
gd_RMSEc      2.309117
ev_RMSEc      2.305043
wrf_RMSEc     6.177237
dtype: float64
min:  morr_RMSEc
max:  wrf_RMSE
2019
morr_RMSE     2.921957
gd_RMSE       2.881760
ev_RMSE       2.840181
wrf_RMSE      6.635490
morr_RMSEc    2.482411
gd_RMSEc      2.480019
ev_RMSEc      2.450300
wrf_RMSEc     6.424498
dtype: float64
min:  ev_RMSEc
max:  wrf_RMSE
2020
morr_RMSE     2.684228
gd_RMSE       2.580255
ev_RMSE       2.574326
wrf_RMSE      7.131511
morr_RMSEc    2.397163
gd_RMSEc      2.317788
ev_RMSEc      2.308801
wrf_RMSEc     6.990024
dtype: float64
min:  ev_RMSEc
max:  wrf_RMSE


#### 1.2 (Weighted) Mean over all years

In [17]:
# get params os all stations of all years of one iveg 
df_years = pd.DataFrame(columns = years, index = ['lambdaGPP_morr', 'radZero_morr', 'alpha_morr', 'beta_morr','lambdaGPP_gd', 'radZero_gd', 'alpha_gd', 'beta_gd','lambdaGPP_ev', 'radZero_ev', 'alpha_ev', 'beta_ev','lambdaGPP_wrf', 'radZero_wrf', 'alpha_wrf', 'beta_wrf']) 
for year in years: 
    df_morris_0, df_gpp_0, df_reco_0 = load_params_as_df(year, 0)
    mean_morris = df_morris_0.mean()
    mean_gpp = df_gpp_0.mean()
    mean_reco = df_reco_0.mean()
        
    df_years[year]['lambdaGPP_morr'] = mean_morris.lambdaGPP
    df_years[year]['radZero_morr'] = mean_morris.radZero
    df_years[year]['alpha_morr'] = mean_morris.alpha
    df_years[year]['beta_morr'] = mean_morris.beta
    
    df_years[year]['lambdaGPP_gd'] = mean_gpp.lambdaGPP_gd
    df_years[year]['radZero_gd'] = mean_gpp.radZero_gd
    df_years[year]['alpha_gd'] = mean_reco.alpha_gd
    df_years[year]['beta_gd'] = mean_reco.beta_ev
    
    df_years[year]['lambdaGPP_ev'] = mean_gpp.lambdaGPP_ev
    df_years[year]['radZero_ev'] = mean_gpp.radZero_ev
    df_years[year]['alpha_ev'] = mean_reco.alpha_ev
    df_years[year]['beta_ev'] = mean_reco.beta_ev
    
    df_years[year]['lambdaGPP_wrf'] = parameters_wrf[1][0]
    df_years[year]['radZero_wrf'] = parameters_wrf[1][1]
    df_years[year]['alpha_wrf'] = parameters_wrf[1][2]
    df_years[year]['beta_wrf'] = parameters_wrf[1][3]
    
    # df_years = df_years[year].append([mean_morris, mean_gpp, mean_reco])

In [18]:
# get mean of params of all years weighted 
df_years['weighted_mean'] = df_years[2017]*0.175 + df_years[2018] * 0.225 + df_years[2019] *0.275 + df_years[2020] *0.325
df_years['mean'] = df_years.mean(axis = 1) 

In [22]:
# predict all years and get rmse 
df_weighted_mean_nee = pd.DataFrame(index = ['FI-Hyy', 'FI-Let', 'SE-Lnn', 'SE-Ros', 'SE-Deg', 'SE-Htm','SE-Nor', 'SE-Svb', 'FI-Qvd'])
for year in years:
    #get stations of that year snames
    station_data = pd.read_csv('Oslo_analysis/Station_evi_lswi_temp_rad_'+str(year)+'.csv', header = 0, index_col=0)
    df_morris = pd.read_csv('Oslo_analysis/VPRMoutput/best_fit_morris_'+str(year)+'.csv', header = None)
    col_list = ['lambdaGPP', 'radZero', 'alpha', 'beta', 'Tmin', 'Tmax', 'Topt', 'Station', 'iveg']
    df_morris.columns = col_list
    df_morris.index = df_morris.Station
    # get stations with same PFT
    df_morris_0 = df_morris[df_morris.iveg == 0]
    snames = df_morris_0.index
    print(snames)
    df_preds = pd.DataFrame()   
    
    for sitename in snames:
        df = get_station_observation_year(StationDataPath, sitename, year)
        #predict GPP 
        lambdaGPP, Tscale, Wscale, Pscale, EVI, Rad, radZero, GPP_or = load_data_gpp(df_morris_0, station_data, sitename, year, StationDataPath)
        gee_morris = GEE_calculate(df_years.weighted_mean.lambdaGPP_morr, Tscale, Wscale, Pscale, EVI, Rad, df_years.weighted_mean.radZero_morr)
        gee_gd = GEE_calculate(df_years.weighted_mean.lambdaGPP_gd, Tscale, Wscale, Pscale, EVI, Rad, df_years.weighted_mean.radZero_gd)
        gee_ev = GEE_calculate(df_years.weighted_mean.lambdaGPP_ev, Tscale, Wscale, Pscale, EVI, Rad, df_years.weighted_mean.radZero_ev)
        gee_wrf = GEE_calculate(df_years.weighted_mean.lambdaGPP_wrf, Tscale, Wscale, Pscale, EVI, Rad,df_years.weighted_mean.radZero_wrf)

        # predict RECO
        reco_or = df['RECO_DT_VUT_REF']    
        t = df['TA_F']    
        reco_morris = reco(df_years.weighted_mean.alpha_morr, df_years.weighted_mean.beta_morr, t)
        reco_gd = reco(df_years.weighted_mean.alpha_gd, df_years.weighted_mean.beta_gd, t) 
        reco_ev = reco(df_years.weighted_mean.alpha_ev, df_years.weighted_mean.beta_ev,t)
        reco_wrf = reco(parameters_wrf[1][2], parameters_wrf[1][3], t)

        #predict NEE params   
        nee_morris = m_NEE(gee_morris, reco_morris)
        nee_gd = m_NEE(gee_gd, reco_gd)
        nee_ev = m_NEE(gee_ev, reco_ev)
        nee_wrf = m_NEE(gee_wrf, reco_wrf)   

        #calculate NEE original
        nee_orc = -df.GPP_DT_VUT_REF + df.RECO_DT_VUT_REF
        #original NEE
        nee_or = df['NEE_VUT_REF']

        #comparison to NEE
        df_weighted_mean_nee.loc[sitename, 'morr_RMSE_'+str(year)] = m_rmse(nee_or, nee_morris)
        
        # df_weighted_mean_nee['morr_RMSE_'+str(year)][sitename] = m_rmse(nee_or, nee_morris)
        # df_weighted_mean_nee.loc[sitename]['gd_RMSE_'+str(year)] = m_rmse(nee_or, nee_gd)
        
        df_weighted_mean_nee.loc[sitename, 'gd_RMSE_'+str(year)] = m_rmse(nee_or, nee_gd)
        df_weighted_mean_nee.loc[sitename, 'ev_RMSE_'+str(year)] =m_rmse(nee_or, nee_ev)
        df_weighted_mean_nee.loc[sitename, 'wrf_RMSE_'+str(year)] = m_rmse(nee_or, nee_wrf) 

        df_weighted_mean_nee.loc[sitename, 'morr_RMSEc_'+str(year)] = m_rmse(nee_orc, nee_morris)
        df_weighted_mean_nee.loc[sitename, 'gd_RMSEc_'+str(year)] = m_rmse(nee_orc, nee_gd)
        df_weighted_mean_nee.loc[sitename, 'ev_RMSEc_'+str(year)] =m_rmse(nee_orc, nee_ev)
        df_weighted_mean_nee.loc[sitename, 'wrf_RMSEc_'+str(year)] = m_rmse(nee_orc, nee_wrf)   
        
        #save predictions in csv file 
        df_predictions_year_mean = pd.DataFrame(index = range(0, len(nee_or)))
        df_predictions_year_mean['Orig_' + sitename + str(year)] = nee_or.reset_index(drop = True)
        df_predictions_year_mean['Orig_cal_' + sitename + str(year)] = nee_orc.reset_index(drop = True)
        df_predictions_year_mean['Morris_' + sitename + str(year)] = nee_morris
        df_predictions_year_mean['Grad_Dec_' + sitename + str(year)] = nee_gd
        df_predictions_year_mean['Diff_Evol_' + sitename+ str(year)] = nee_ev
        df_predictions_year_mean['WRF_' + sitename+ str(year)] =nee_wrf
                        
        df_preds = pd.concat([df_preds, df_predictions_year_mean], axis=1)        
    df_preds.to_csv('Oslo_analysis/VPRMoutput/Predictions_Observations_weighted_mean' + str(year) + '.csv')
        
        
        

Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')
Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Svb', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')
Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Svb', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')
Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Svb', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')


In [48]:
df_weighted_mean_nee

,morr_RMSE_2017,gd_RMSE_2017,ev_RMSE_2017,wrf_RMSE_2017,morr_RMSEc_2017,gd_RMSEc_2017,ev_RMSEc_2017,wrf_RMSEc_2017,morr_RMSE_2018,gd_RMSE_2018,...,ev_RMSEc_2019,wrf_RMSEc_2019,morr_RMSE_2020,gd_RMSE_2020,ev_RMSE_2020,wrf_RMSE_2020,morr_RMSEc_2020,gd_RMSEc_2020,ev_RMSEc_2020,wrf_RMSEc_2020
FI-Hyy,2.599725,2.678340,2.668445,6.353097,2.301491,2.371968,2.358891,6.289728,2.024513,2.056130,...,1.482749,7.078029,2.487610,2.424500,2.384900,8.746080,2.402494,2.362063,2.315897,8.629661
FI-Let,3.313728,3.468418,3.439877,5.324334,2.935417,2.965586,2.946560,5.290518,2.846208,2.719629,...,2.009968,5.803283,2.616444,2.457564,2.437385,7.644608,2.626396,2.534798,2.501433,7.636839
SE-Lnn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SE-Ros,1.814991,1.967846,1.950298,4.787674,1.710976,1.859776,1.840244,4.732832,1.816828,2.137011,...,1.989604,5.230324,1.393325,1.509097,1.500442,5.293395,1.508697,1.683542,1.664898,5.187018
SE-Deg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SE-Htm,5.874239,6.006254,6.007167,6.997457,5.286398,5.374011,5.378119,6.572246,4.103646,3.860125,...,3.900077,8.211636,4.790270,4.526036,4.548492,8.391202,4.197249,4.127523,4.121586,8.144634
SE-Nor,3.341896,3.036074,3.053244,6.584679,2.296795,2.158464,2.155843,5.897136,2.963498,2.610158,...,3.340686,7.422547,3.410919,2.997082,3.023824,7.713231,3.105980,2.767015,2.780998,7.607207
SE-Svb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.259141,2.444222,...,1.978609,4.801169,1.976531,1.963951,1.956079,5.000550,1.379673,1.496838,1.479835,4.734785
FI-Qvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_weighted_mean_nee.filter(regex='2017').mean()

morr_RMSE_2017     3.388916
gd_RMSE_2017       3.431386
ev_RMSE_2017       3.423806
wrf_RMSE_2017      6.009448
morr_RMSEc_2017    2.906216
gd_RMSEc_2017      2.945961
ev_RMSEc_2017      2.935932
wrf_RMSEc_2017     5.756492
dtype: float64

In [26]:
df_weighted_mean_nee.filter(regex='2018').mean()

morr_RMSE_2018     2.668972
gd_RMSE_2018       2.637879
ev_RMSE_2018       2.624009
wrf_RMSE_2018      6.445116
morr_RMSEc_2018    2.316546
gd_RMSEc_2018      2.336234
ev_RMSEc_2018      2.314973
wrf_RMSEc_2018     6.177237
dtype: float64

In [27]:
df_weighted_mean_nee.filter(regex='2019').mean()

morr_RMSE_2019     2.927161
gd_RMSE_2019       2.835016
ev_RMSE_2019       2.836587
wrf_RMSE_2019      6.635490
morr_RMSEc_2019    2.487179
gd_RMSEc_2019      2.461115
ev_RMSEc_2019      2.450282
wrf_RMSEc_2019     6.424498
dtype: float64

In [28]:
df_weighted_mean_nee.filter(regex='2020').mean()

morr_RMSE_2020     2.779183
gd_RMSE_2020       2.646372
ev_RMSE_2020       2.641854
wrf_RMSE_2020      7.131511
morr_RMSEc_2020    2.536748
gd_RMSEc_2020      2.495297
ev_RMSEc_2020      2.477441
wrf_RMSEc_2020     6.990024
dtype: float64

### 2. Combine and compare parameters for same PFT no winter edition

#### 2.1 Mean over all stations (specific year) 

In [39]:
# predict reco with all 4 versions 
#compare to 0riginal 
year = 2017
df_morris_0, df_gpp_0, df_reco_0 = load_params_as_df(year, 0, winter = True)
mean_morris = df_morris_0.mean()
mean_gpp = df_gpp_0.mean()
mean_reco = df_reco_0.mean()

snames = df_morris_0.index
df_mean_reco = pd.DataFrame(index = snames, columns = ['morr_RMSE', 'gd_RMSE','ev_RMSE', 'wrf_RMSE'])

for sitename in snames: 
    df = get_station_observation_year(StationDataPath, sitename, year)
    df.date = pd.to_datetime(df.date)
    df['month'] = df['date'].dt.month
    df = df[df['month'] != 12]
    df = df[df['month'] != 2]
    df = df[df['month'] != 1]
    
    reco_or = df['RECO_DT_VUT_REF']    
    t = df['TA_F']
    
    reco_morris = reco(mean_morris.alpha, mean_morris.beta, t)
    reco_gd = reco(mean_reco.alpha_gd, mean_reco.beta_gd, t) 
    reco_ev = reco(mean_reco.alpha_ev, mean_reco.beta_gd,t)
    reco_wrf = reco(parameters_wrf[1][2], parameters_wrf[1][3], t)
    
    df_mean_reco.loc[sitename]['morr_RMSE'] = m_rmse(reco_or, reco_morris)
    df_mean_reco.loc[sitename]['gd_RMSE'] = m_rmse(reco_or, reco_gd)
    df_mean_reco.loc[sitename]['ev_RMSE'] =m_rmse(reco_or, reco_ev)
    df_mean_reco.loc[sitename]['wrf_RMSE'] = m_rmse(reco_or, reco_wrf) 
    
    # print('Morris :', rmse_morris, 'GD: ', rmse_gd, 'EV:  ', rmse_ev, 'wrf: ', rmse_wrf)

In [40]:
df_mean_reco.mean()

morr_RMSE    2.509959
gd_RMSE      1.909734
ev_RMSE      1.908079
wrf_RMSE     3.114778
dtype: float64

In [53]:
# predict GPP #compare to 0riginal 
df_mean_gpp = pd.DataFrame(index = snames, columns = ['morr_RMSE', 'gd_RMSE','ev_RMSE', 'wrf_RMSE'])
station_data = pd.read_csv('Oslo_analysis/Station_evi_lswi_temp_rad_'+str(year)+'.csv', header = 0, index_col=0)
station_data.loc[:,'month'] = pd.to_datetime(station_data.index).month.to_list()
station_data = station_data[station_data['month']!= 12]
station_data = station_data[station_data['month']!= 1]
station_data = station_data[station_data['month']!= 2]

for sitename in snames: 
    # df = get_station_observation_year(StationDataPath, sitename, year)
    
    lambdaGPP, Tscale, Wscale, Pscale, EVI, Rad, radZero, GPP_or = load_data_gpp(df_morris_0, station_data, sitename, year, StationDataPath, months = True)
    
    gee_morris = GEE_calculate(mean_morris.lambdaGPP, Tscale, Wscale, Pscale, EVI, Rad, mean_morris.radZero)
    gee_gd = GEE_calculate(mean_gpp.lambdaGPP_gd, Tscale, Wscale, Pscale, EVI, Rad, mean_gpp.radZero_gd)
    gee_ev = GEE_calculate(mean_gpp.lambdaGPP_ev, Tscale, Wscale, Pscale, EVI, Rad, mean_gpp.radZero_ev)
    gee_wrf = GEE_calculate(parameters_wrf[1][0], Tscale, Wscale, Pscale, EVI, Rad, parameters_wrf[1][1])

    df_mean_gpp.loc[sitename]['morr_RMSE'] = m_rmse(GPP_or, gee_morris)
    df_mean_gpp.loc[sitename]['gd_RMSE'] = m_rmse(GPP_or, gee_gd)
    df_mean_gpp.loc[sitename]['ev_RMSE'] =m_rmse(GPP_or, gee_ev)
    df_mean_gpp.loc[sitename]['wrf_RMSE'] = m_rmse(GPP_or, gee_wrf) 

In [43]:
df_mean_gpp.mean()

morr_RMSE    3.780909
gd_RMSE      3.937283
ev_RMSE      3.868797
wrf_RMSE     5.781853
dtype: float64

In [49]:
# predict NEE 
for year in years:
    df_preds, df_mean_nee = predict_NEE(year, StationDataPath, parameters_wrf, 0, winter =True)
    print(year)
    print(df_mean_nee.mean())
    print('min: ',df_mean_nee.mean().idxmin())
    print('max: ',df_mean_nee.mean().idxmax())

2017
morr_RMSE     3.737147
gd_RMSE       3.771303
ev_RMSE       3.753087
wrf_RMSE      6.893103
morr_RMSEc    3.306899
gd_RMSEc      3.362373
ev_RMSEc      3.318866
wrf_RMSEc     6.642666
dtype: float64
min:  morr_RMSEc
max:  wrf_RMSE
2018
morr_RMSE     2.874744
gd_RMSE       2.831971
ev_RMSE       2.841239
wrf_RMSE      7.186745
morr_RMSEc    2.478499
gd_RMSEc      2.453034
ev_RMSEc      2.505348
wrf_RMSEc     6.899676
dtype: float64
min:  gd_RMSEc
max:  wrf_RMSE
2019
morr_RMSE     3.191355
gd_RMSE       3.095420
ev_RMSE       3.088817
wrf_RMSE      7.703095
morr_RMSEc    2.754331
gd_RMSEc      2.671178
ev_RMSEc      2.674853
wrf_RMSEc     7.504105
dtype: float64
min:  gd_RMSEc
max:  wrf_RMSE
2020
morr_RMSE     3.012540
gd_RMSE       2.901385
ev_RMSE       2.900057
wrf_RMSE      8.449305
morr_RMSEc    2.727233
gd_RMSEc      2.629409
ev_RMSEc      2.620852
wrf_RMSEc     8.329781
dtype: float64
min:  ev_RMSEc
max:  wrf_RMSE


#### 2.2 (Weighted) Mean over all years

In [50]:
# get params os all stations of all years of one iveg 
df_years = pd.DataFrame(columns = years, index = ['lambdaGPP_morr', 'radZero_morr', 'alpha_morr', 'beta_morr','lambdaGPP_gd', 'radZero_gd', 'alpha_gd', 'beta_gd','lambdaGPP_ev', 'radZero_ev', 'alpha_ev', 'beta_ev','lambdaGPP_wrf', 'radZero_wrf', 'alpha_wrf', 'beta_wrf']) 
for year in years: 
    df_morris_0, df_gpp_0, df_reco_0 = load_params_as_df(year, 0, winter = True)
    mean_morris = df_morris_0.mean()
    mean_gpp = df_gpp_0.mean()
    mean_reco = df_reco_0.mean()
        
    df_years[year]['lambdaGPP_morr'] = mean_morris.lambdaGPP
    df_years[year]['radZero_morr'] = mean_morris.radZero
    df_years[year]['alpha_morr'] = mean_morris.alpha
    df_years[year]['beta_morr'] = mean_morris.beta
    
    df_years[year]['lambdaGPP_gd'] = mean_gpp.lambdaGPP_gd
    df_years[year]['radZero_gd'] = mean_gpp.radZero_gd
    df_years[year]['alpha_gd'] = mean_reco.alpha_gd
    df_years[year]['beta_gd'] = mean_reco.beta_ev
    
    df_years[year]['lambdaGPP_ev'] = mean_gpp.lambdaGPP_ev
    df_years[year]['radZero_ev'] = mean_gpp.radZero_ev
    df_years[year]['alpha_ev'] = mean_reco.alpha_ev
    df_years[year]['beta_ev'] = mean_reco.beta_ev
    
    df_years[year]['lambdaGPP_wrf'] = parameters_wrf[1][0]
    df_years[year]['radZero_wrf'] = parameters_wrf[1][1]
    df_years[year]['alpha_wrf'] = parameters_wrf[1][2]
    df_years[year]['beta_wrf'] = parameters_wrf[1][3]

In [51]:
# get mean of params of all years weighted 
df_years['weighted_mean'] = df_years[2017]*0.175 + df_years[2018] * 0.225 + df_years[2019] *0.275 + df_years[2020] *0.325
df_years['mean'] = df_years.mean(axis = 1) 

In [56]:
# predict all years and get rmse 
df_weighted_mean_nee = pd.DataFrame(index = ['FI-Hyy', 'FI-Let', 'SE-Lnn', 'SE-Ros', 'SE-Deg', 'SE-Htm','SE-Nor', 'SE-Svb', 'FI-Qvd'])
for year in years:
    #get stations of that year snames
    df_morris = pd.read_csv('Oslo_analysis/VPRMoutput/best_fit_morris_nowinter_'+str(year)+'.csv', header = None)

    station_data = pd.read_csv('Oslo_analysis/Station_evi_lswi_temp_rad_'+str(year)+'.csv', header = 0, index_col=0)
    station_data = pd.read_csv('Oslo_analysis/Station_evi_lswi_temp_rad_'+str(year)+'.csv', header = 0, index_col=0)
    station_data.loc[:,'month'] = pd.to_datetime(station_data.index).month.to_list()
    station_data = station_data[station_data['month']!= 12]
    station_data = station_data[station_data['month']!= 1]
    station_data = station_data[station_data['month']!= 2]
    
    col_list = ['lambdaGPP', 'radZero', 'alpha', 'beta', 'Tmin', 'Tmax', 'Topt', 'Station', 'iveg']
    df_morris.columns = col_list
    df_morris.index = df_morris.Station
    # get stations with same PFT
    df_morris_0 = df_morris[df_morris.iveg == 0]
    snames = df_morris_0.index
    print(snames)
    df_preds = pd.DataFrame()   
    
    for sitename in snames:
        df = get_station_observation_year(StationDataPath, sitename, year)
        df.date = pd.to_datetime(df.date)
        df['month'] = df['date'].dt.month
        df = df[df['month'] != 12]
        df = df[df['month'] != 2]
        df = df[df['month'] != 1]
        #predict GPP 
        lambdaGPP, Tscale, Wscale, Pscale, EVI, Rad, radZero, GPP_or = load_data_gpp(df_morris_0, station_data, sitename, year, StationDataPath, months = True)
        gee_morris = GEE_calculate(df_years.weighted_mean.lambdaGPP_morr, Tscale, Wscale, Pscale, EVI, Rad, df_years.weighted_mean.radZero_morr)
        gee_gd = GEE_calculate(df_years.weighted_mean.lambdaGPP_gd, Tscale, Wscale, Pscale, EVI, Rad, df_years.weighted_mean.radZero_gd)
        gee_ev = GEE_calculate(df_years.weighted_mean.lambdaGPP_ev, Tscale, Wscale, Pscale, EVI, Rad, df_years.weighted_mean.radZero_ev)
        gee_wrf = GEE_calculate(df_years.weighted_mean.lambdaGPP_wrf, Tscale, Wscale, Pscale, EVI, Rad,df_years.weighted_mean.radZero_wrf)

        # predict RECO
        reco_or = df['RECO_DT_VUT_REF']    
        t = df['TA_F']    
        reco_morris = reco(df_years.weighted_mean.alpha_morr, df_years.weighted_mean.beta_morr, t)
        reco_gd = reco(df_years.weighted_mean.alpha_gd, df_years.weighted_mean.beta_gd, t) 
        reco_ev = reco(df_years.weighted_mean.alpha_ev, df_years.weighted_mean.beta_ev,t)
        reco_wrf = reco(parameters_wrf[1][2], parameters_wrf[1][3], t)

        #predict NEE params   
        nee_morris = m_NEE(gee_morris, reco_morris)
        nee_gd = m_NEE(gee_gd, reco_gd)
        nee_ev = m_NEE(gee_ev, reco_ev)
        nee_wrf = m_NEE(gee_wrf, reco_wrf)   

        #calculate NEE original
        nee_orc = -df.GPP_DT_VUT_REF + df.RECO_DT_VUT_REF
        #original NEE
        nee_or = df['NEE_VUT_REF']

        #comparison to NEE
        df_weighted_mean_nee.loc[sitename, 'morr_RMSE_'+str(year)] = m_rmse(nee_or, nee_morris)
        
        # df_weighted_mean_nee['morr_RMSE_'+str(year)][sitename] = m_rmse(nee_or, nee_morris)
        # df_weighted_mean_nee.loc[sitename]['gd_RMSE_'+str(year)] = m_rmse(nee_or, nee_gd)
        
        df_weighted_mean_nee.loc[sitename, 'gd_RMSE_'+str(year)] = m_rmse(nee_or, nee_gd)
        df_weighted_mean_nee.loc[sitename, 'ev_RMSE_'+str(year)] =m_rmse(nee_or, nee_ev)
        df_weighted_mean_nee.loc[sitename, 'wrf_RMSE_'+str(year)] = m_rmse(nee_or, nee_wrf) 

        df_weighted_mean_nee.loc[sitename, 'morr_RMSEc_'+str(year)] = m_rmse(nee_orc, nee_morris)
        df_weighted_mean_nee.loc[sitename, 'gd_RMSEc_'+str(year)] = m_rmse(nee_orc, nee_gd)
        df_weighted_mean_nee.loc[sitename, 'ev_RMSEc_'+str(year)] =m_rmse(nee_orc, nee_ev)
        df_weighted_mean_nee.loc[sitename, 'wrf_RMSEc_'+str(year)] = m_rmse(nee_orc, nee_wrf)   
        
        #save predictions in csv file 
        df_predictions_year_mean = pd.DataFrame(index = range(0, len(nee_or)))
        df_predictions_year_mean['Orig_' + sitename + str(year)] = nee_or.reset_index(drop = True)
        df_predictions_year_mean['Orig_cal_' + sitename + str(year)] = nee_orc.reset_index(drop = True)
        df_predictions_year_mean['Morris_' + sitename + str(year)] = nee_morris
        df_predictions_year_mean['Grad_Dec_' + sitename + str(year)] = nee_gd
        df_predictions_year_mean['Diff_Evol_' + sitename+ str(year)] = nee_ev
        df_predictions_year_mean['WRF_' + sitename+ str(year)] =nee_wrf
                        
        df_preds = pd.concat([df_preds, df_predictions_year_mean], axis=1)        
    df_preds.to_csv('Oslo_analysis/VPRMoutput/Predictions_Observations_weighted_mean_nowinter_' + str(year) + '.csv')
        

Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')
Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Svb', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')
Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Svb', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')
Index(['FI-Hyy', 'FI-Let', 'SE-Ros', 'SE-Svb', 'SE-Htm', 'SE-Nor'], dtype='object', name='Station')


In [57]:
df_weighted_mean_nee

,morr_RMSE_2017,gd_RMSE_2017,ev_RMSE_2017,wrf_RMSE_2017,morr_RMSEc_2017,gd_RMSEc_2017,ev_RMSEc_2017,wrf_RMSEc_2017,morr_RMSE_2018,gd_RMSE_2018,...,ev_RMSEc_2019,wrf_RMSEc_2019,morr_RMSE_2020,gd_RMSE_2020,ev_RMSE_2020,wrf_RMSE_2020,morr_RMSEc_2020,gd_RMSEc_2020,ev_RMSEc_2020,wrf_RMSEc_2020
FI-Hyy,2.907622,2.993990,2.978020,7.250628,2.616196,2.674581,2.662153,7.233957,2.151222,2.181232,...,1.500725,8.311251,2.962431,2.806164,2.825920,10.858290,2.908277,2.760347,2.778783,10.743069
FI-Let,3.779298,3.949402,3.931419,6.009550,3.321658,3.351342,3.342230,5.978832,3.115717,2.907354,...,2.283464,7.236236,3.095330,2.853142,2.879141,9.296188,3.189226,2.986598,3.012086,9.332810
SE-Lnn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SE-Ros,1.943119,2.081133,2.061273,5.638246,1.846783,1.982735,1.962638,5.572832,1.843574,2.179973,...,1.949730,5.642123,1.446300,1.579018,1.562905,6.182056,1.648246,1.817435,1.800202,6.076653
SE-Deg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SE-Htm,6.365128,6.518018,6.499051,7.677858,5.727456,5.835171,5.817938,7.224534,4.526003,4.202746,...,4.321704,9.227097,5.304090,4.992606,5.007059,9.380542,4.632696,4.459954,4.473337,9.146834
SE-Nor,3.706723,3.397409,3.415631,7.889235,2.619795,2.459508,2.473005,7.203174,3.257363,2.906375,...,4.062818,9.325758,3.935666,3.484938,3.519277,9.328312,3.674806,3.269155,3.307296,9.283875
SE-Svb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.319276,2.529595,...,2.016650,5.282166,2.172543,2.198302,2.183109,5.650440,1.554893,1.693393,1.673771,5.395446
FI-Qvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df_weighted_mean_nee.filter(regex='2017').mean()

morr_RMSE_2017     3.740378
gd_RMSE_2017       3.787990
ev_RMSE_2017       3.777079
wrf_RMSE_2017      6.893103
morr_RMSEc_2017    3.226378
gd_RMSEc_2017      3.260668
ev_RMSEc_2017      3.251593
wrf_RMSEc_2017     6.642666
dtype: float64

In [59]:
df_weighted_mean_nee.filter(regex='2018').mean()

morr_RMSE_2018     2.868859
gd_RMSE_2018       2.817879
ev_RMSE_2018       2.812544
wrf_RMSE_2018      7.186745
morr_RMSEc_2018    2.465458
gd_RMSEc_2018      2.445053
ev_RMSEc_2018      2.439780
wrf_RMSEc_2018     6.899676
dtype: float64

In [60]:
df_weighted_mean_nee.filter(regex='2019').mean()

morr_RMSE_2019     3.206981
gd_RMSE_2019       3.076415
ev_RMSE_2019       3.081259
wrf_RMSE_2019      7.703095
morr_RMSEc_2019    2.773601
gd_RMSEc_2019      2.685109
ev_RMSEc_2019      2.689182
wrf_RMSEc_2019     7.504105
dtype: float64

In [61]:
df_weighted_mean_nee.filter(regex='2020').mean()

morr_RMSE_2020     3.152727
gd_RMSE_2020       2.985695
ev_RMSE_2020       2.996235
wrf_RMSE_2020      8.449305
morr_RMSEc_2020    2.934691
gd_RMSEc_2020      2.831147
ev_RMSEc_2020      2.840913
wrf_RMSEc_2020     8.329781
dtype: float64